More recent models, such as TSMixer, TFT and NHITS achieve better accuracy than LSTM in most settings.

In [1]:
import logging
import pandas as pd
from ray import tune
from neuralforecast import NeuralForecast
from neuralforecast.auto import (
    AutoNHITS, AutoNBEATS, AutoNBEATSx, AutoTFT, AutoVanillaTransformer,
    AutoInformer, AutoAutoformer, AutoFEDformer, AutoPatchTST, AutoLSTM, AutoGRU,
    AutoRNN, AutoDilatedRNN, AutoTCN, AutoMLP, AutoMLPMultivariate, AutoDLinear,
    AutoNLinear, AutoTimeXer, AutoTSMixer, AutoTSMixerx, AutoTimeMixer, AutoTimeLLM,
    AutoTimesNet, AutoiTransformer, AutoSOFTS, AutoStemGNN, AutoBiTCN,
    AutoDeepAR, AutoDeepNPTS, AutoReversibleMixtureOfKAN, AutoHINT
)
from neuralforecast.utils import AirPassengersDF
from utilsforecast.plotting import plot_series

# Suppress unnecessary logging
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)

2025-03-19 18:40:40,501	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-19 18:40:40,761	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ImportError: cannot import name 'AutoTimeLLM' from 'neuralforecast.auto' (c:\Users\PC314\anaconda3\Lib\site-packages\neuralforecast\auto.py)

In [ ]:


# Load dataset
Y_df = AirPassengersDF
horizon = 12  # Forecasting horizon

# List of AutoModels
auto_models = [
    AutoNHITS, AutoNBEATS, AutoNBEATSx, AutoTFT, AutoVanillaTransformer,
    AutoInformer, AutoAutoformer, AutoFEDformer, AutoPatchTST, AutoLSTM, AutoGRU,
    AutoRNN, AutoDilatedRNN, AutoTCN, AutoMLP, AutoMLPMultivariate, AutoDLinear,
    AutoNLinear, AutoTimeXer, AutoTSMixer, AutoTSMixerx, AutoTimeMixer, AutoTimeLLM,
    AutoTimesNet, AutoiTransformer, AutoSOFTS, AutoStemGNN, AutoBiTCN,
    AutoDeepAR, AutoDeepNPTS, AutoReversibleMixtureOfKAN, AutoHINT
]

# Initialize models with default hyperparameter configurations
def get_auto_model(model_cls):
    config = model_cls.get_default_config(h=horizon, backend="ray")
    config["random_seed"] = tune.randint(1, 10)
    return model_cls(h=horizon, config=config, backend='ray', num_samples=3)

# Create models
models = [get_auto_model(model_cls) for model_cls in auto_models]

# Initialize and fit NeuralForecast
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=Y_df)

# Generate predictions
Y_hat_df = nf.predict()

# Display predictions
plot_series(Y_df, Y_hat_df)

# Save results
Y_hat_df.to_csv("forecast_results.csv", index=False)
print("Forecasting completed. Results saved in forecast_results.csv")
